# Symbolic neural networks for cognitive capacities
[Paper](http://reason.cs.uiuc.edu/tsvi/BICA_93_Main.pdf)

In [1]:
%run achler_functions.py

# Learn Iris dataset instances

In [2]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
exp = np.zeros( (len(iris.data), len(iris.feature_names) + len(iris.target_names)) )
exp_labels = []
for index, item in enumerate(iris.data):
    instance = {}
    instance = {iris.target_names[iris.target[index]]: 1}
    for i in range(len(iris.feature_names)):
        instance[iris.feature_names[i]] = iris.data[index][i]
    exp, exp_labels = learn_instance([instance,], exp, exp_labels)
print(exp.shape)
print(exp)
print(exp_labels)

(150, 7)
[[ 0.2  1.4  5.1 ...,  1.   0.   0. ]
 [ 0.2  1.4  4.9 ...,  1.   0.   0. ]
 [ 0.2  1.3  4.7 ...,  1.   0.   0. ]
 ..., 
 [ 2.   5.2  6.5 ...,  0.   0.   1. ]
 [ 2.3  5.4  6.2 ...,  0.   0.   1. ]
 [ 1.8  5.1  5.9 ...,  0.   0.   1. ]]
['petal width (cm)', 'petal length (cm)', 'sepal length (cm)', 'sepal width (cm)', 'setosa', 'versicolor', 'virginica']


## Make categories of the Iris types

In [3]:
def make_categories(category_list, exp, exp_labels):
    categories = {}
    for category in category_list:
        if category in exp_labels:
            category_index = exp_labels.index(category)
            categories[category] = np.matrix(np.zeros(exp.shape[1]))
            for index in range(exp.shape[0]):
                if exp.item( (index, category_index) ) != 0:
                    categories[category] += exp[index, :]
    return categories
categories = make_categories(iris.target_names, exp, exp_labels)
categories

{'setosa': matrix([[  12.2,   73.2,  250.3,  170.9,   50. ,    0. ,    0. ]]),
 'versicolor': matrix([[  66.3,  213. ,  296.8,  138.5,    0. ,   50. ,    0. ]]),
 'virginica': matrix([[ 101.3,  277.6,  329.4,  148.7,    0. ,    0. ,   50. ]])}

Do the categories work?

In [4]:
pinv = np.linalg.pinv(exp)
pinv_transpose = pinv.transpose()
prediction_setosa = pinv_transpose * categories["setosa"].T
hit = 0
miss = 0
for i in range(len(iris.data)):
    if iris.target[i] == 0:
        if round( prediction_setosa.item( (i, 0) ), 2) == 1.0:
            hit += 1
        else:
            print(i, prediction_setosa.item( (i, 0) ), "<>", iris.target[i])
            miss += 1
print("{}/{}".format(hit, hit + miss))

50/50


In [5]:
prediction_versicolor = pinv_transpose * categories["versicolor"].T
hit = 0
miss = 0
for i in range(len(iris.data)):
    if iris.target[i] == 1:
        if round( prediction_versicolor.item( (i, 0) ), 2) == 1.0:
            hit += 1
        else:
            print(i, prediction_versicolor.item( (i, 0) ), "<>", iris.target[i])
            miss += 1
print("{}/{}".format(hit, hit + miss))

50/50


In [6]:
prediction_virginica = pinv_transpose * categories["virginica"].T
hit = 0
miss = 0
for i in range(len(iris.data)):
    if iris.target[i] == 2:
        if round( prediction_virginica.item( (i, 0) ), 2) == 1.0:
            hit += 1
        else:
            print(i, prediction_virginica.item( (i, 0) ), "<>", iris.target[i])
            miss += 1
print("{}/{}".format(hit, hit + miss))

50/50


How does some new instance get classified?

In [7]:
iris.data[17]

array([ 5.1,  3.5,  1.4,  0.3])

In [8]:
iris.target_names[iris.target[17]]

'setosa'

In [9]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
instance = {'sepal length (cm)' : 5.0, # a tweak on instance 17
 'sepal width (cm)': 3.5,
 'petal length (cm)': 1.4,
 'petal width (cm)': 0.3}

In [11]:
instance_vector = vectorize(instance, exp, exp_labels)
instance_vector[0]

matrix([[ 0.3,  1.4,  5. ,  3.5,  0. ,  0. ,  0. ]])

In [12]:
prediction = pinv_transpose * instance_vector[0].T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.26


'setosa'

Try missing out a property:

In [13]:
instance = {'sepal width (cm)': 3.5,
 'petal length (cm)': 1.4,
 'petal width (cm)': 0.3}

In [14]:
instance_vector = vectorize(instance, exp, exp_labels)
instance_vector[0]

matrix([[ 0.3,  1.4,  0. ,  3.5,  0. ,  0. ,  0. ]])

In [15]:
prediction = pinv_transpose * instance_vector[0].T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.24


'setosa'

Try missing out a different property:

In [16]:
instance = {'sepal length (cm)' : 5.1,
 'petal length (cm)': 1.4,
 'petal width (cm)': 0.3}

In [17]:
instance_vector = vectorize(instance, exp, exp_labels)
instance_vector[0]

matrix([[ 0.3,  1.4,  5.1,  0. ,  0. ,  0. ,  0. ]])

In [18]:
prediction = pinv_transpose * instance_vector[0].T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.25


'virginica'

Try just two distiguishing properties:

In [19]:
instance = {'petal length (cm)': 1.4,
 'petal width (cm)': 0.3}

In [20]:
instance_vector = vectorize(instance, exp, exp_labels)
instance_vector[0]

matrix([[ 0.3,  1.4,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [21]:
prediction = pinv_transpose * instance_vector[0].T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.09


'versicolor'

How about the "average setosa"?

In [22]:
categories["setosa"] / 50

matrix([[ 0.244,  1.464,  5.006,  3.418,  1.   ,  0.   ,  0.   ]])

In [23]:
prediction = pinv_transpose * (categories["setosa"] / 50).T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.02


'setosa'

How about the "average virginica"?

In [24]:
categories["virginica"] / 50

matrix([[ 2.026,  5.552,  6.588,  2.974,  0.   ,  0.   ,  1.   ]])

In [25]:
prediction = pinv_transpose * (categories["virginica"] / 50).T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.02


'virginica'

How about the "average versicolor"?

In [26]:
avg_versicolor = categories["versicolor"] / 50
avg_versicolor

matrix([[ 1.326,  4.26 ,  5.936,  2.77 ,  0.   ,  1.   ,  0.   ]])

In [27]:
prediction = pinv_transpose * avg_versicolor.T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.02


'versicolor'

Unlabled doesn't work, however:

In [28]:
instance = {'sepal length (cm)' : avg_versicolor.item( 0, exp_labels.index('sepal length (cm)') ),
 'sepal width (cm)' : avg_versicolor.item( 0, exp_labels.index('sepal width (cm)') ),
 'petal length (cm)' : avg_versicolor.item( 0, exp_labels.index('petal length (cm)') ),
 'petal width (cm)' : avg_versicolor.item( 0, exp_labels.index('petal width (cm)') )}
instance

{'petal length (cm)': 4.26,
 'petal width (cm)': 1.3259999999999998,
 'sepal length (cm)': 5.936,
 'sepal width (cm)': 2.7700000000000005}

In [29]:
instance_vector = vectorize(instance, exp, exp_labels)
instance_vector[0]

matrix([[ 1.326,  4.26 ,  5.936,  2.77 ,  0.   ,  0.   ,  0.   ]])

In [30]:
prediction = pinv_transpose * instance_vector[0].T
print(round(prediction.max(),2))
iris.target_names[iris.target.item( prediction.argmax() )]

0.21


'virginica'